In [1]:
import pandas as pd
df = pd.read_csv('data/stock2008-23.csv')
#Calculating the daily change, removing what seems to be unnecessary columns
df['change'] = df['Close'] - df['Open']
df['change_pct'] = df['change'] *100/ df['Open']
df=df.drop(columns=['Open','High','Low','Close', 'Adj Close', 'Volume'])

In [3]:
import pandas as pd

# Get unique tickers
unique_tickers = df['Ticker'].unique()

# Loop through each unique ticker
for ticker in unique_tickers:
    # Filter data for the current ticker
    ticker_data = df[df['Ticker'] == ticker][['Date', 'change_pct']].rename(columns={'change_pct': 'change'})

    # Function to calculate upper and lower bounds
    def calculate_bounds(group):
        non_ticker = group[group['Ticker'] != ticker]['change_pct']
        mean = non_ticker.mean()
        std = non_ticker.std()
        return pd.Series({'mean': mean, 'std': std})

    # Group by Date to compute bounds
    bounds = df.groupby('Date').apply(calculate_bounds).reset_index()

    # Merge the ticker data with the bounds
    df_alt_ticker = pd.merge(ticker_data, bounds, left_on='Date', right_on='Date')

    # Calculate the effect column
    def classify_effect(row):
        if row['mean'] - row['std'] <= row['change'] <= row['mean'] + row['std']:
            return 0
        elif row['mean'] - 2 * row['std'] <= row['change'] < row['mean'] - row['std'] or \
             row['mean'] + row['std'] < row['change'] <= row['mean'] + 2 * row['std']:
            return 1
        elif row['mean'] - 3 * row['std'] <= row['change'] < row['mean'] - 2 * row['std'] or \
             row['mean'] + 2 * row['std'] < row['change'] <= row['mean'] + 3 * row['std']:
            return 2
        else:
            return 3

    df_alt_ticker['effect'] = df_alt_ticker.apply(classify_effect, axis=1)

    # Final dataframe with required columns
    df_alt_ticker = df_alt_ticker[['Date', 'change', 'mean', 'std', 'effect']]

    # Create specific file for the current ticker
    df_alt_ticker.to_csv(f'data/test//multi_log_reg/multi_{ticker.replace("^", "")}.csv', index=False)

C:\Users\SJANO\AppData\Local\Temp\ipykernel_8084\718546351.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bounds = df.groupby('Date').apply(calculate_bounds).reset_index()
C:\Users\SJANO\AppData\Local\Temp\ipykernel_8084\718546351.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bounds = df.groupby('Date').apply(calculate_bounds).reset_index()
C:\Users\SJANO\AppData\Local\Temp\ipykernel_8084\71854